In [ ]:
import skimage
import cv2
import numpy as np
from matplotlib import pyplot as plt
import sys

In [ ]:
import pandas as pd
dataset = pd.read_excel("/kaggle/input/miniddsm2/MINI-DDSM-Complete-JPEG-8/DataWMask.xlsx")
dataset['fullPath'] = dataset['fullPath'].apply(lambda x: x.replace("\\"," ").replace(" ", "/"))
dataset['Tumour_Contour'] = dataset['Tumour_Contour'].apply(lambda x: x.replace("\\"," ").replace(" ", "/"))
dataset

In [ ]:
name_image = dataset['fullPath'].apply(lambda x: x.split('/')[-1])
name_mask = dataset['Tumour_Contour'].apply(lambda x: x.split('/')[-1])

In [ ]:
!ls /kaggle/working/

In [ ]:
import os

os.mkdir('mask')
os.mkdir('image')

CẮT ẢNH

In [ ]:
name_image_mask = name_image.apply(lambda x: x.split('.jpg')[0])
name_image_mask

In [ ]:
cd /kaggle/input/miniddsm2/MINI-DDSM-Complete-JPEG-8

In [ ]:
# read image
for i in range(0,1000):
    
    img = skimage.io.imread(dataset['fullPath'][i],as_gray=True)
#     img = cv2.resize(img, (int(np.round(img.shape[1]/10)), int(np.round(img.shape[0]/10))))

    # resize image
    img = img[100:]
    hh, ww = img.shape[:2]

    # convert to grayscale

    # apply otsu thresholding
    thresh = cv2.threshold(img, 0, 255, cv2.THRESH_OTSU )[1] 

    # apply morphology close to remove small regions
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
    morph = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

    # apply morphology open to separate breast from other regions
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
    morph = cv2.morphologyEx(morph, cv2.MORPH_OPEN, kernel)

    # get largest contour
    contours = cv2.findContours(morph, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = contours[0] if len(contours) == 2 else contours[1]
    big_contour = max(contours, key=cv2.contourArea)

    # draw largest contour as white filled on black background as mask
    mask = np.zeros((hh,ww), dtype=np.uint8)
    cv2.drawContours(mask, [big_contour], 0, 255, cv2.FILLED)

    # dilate mask
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (500,500))
    mask = cv2.morphologyEx(mask, cv2.MORPH_DILATE, kernel)

    # apply mask to image
    result = cv2.bitwise_and(img, img, mask=mask)

    # save results
    original = mask.copy()
    blur = cv2.GaussianBlur(mask, (9,9), 0)
    thresh = cv2.threshold(mask,0,255,cv2.THRESH_OTSU + cv2.THRESH_BINARY)[1]

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2,2))
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
    dilate_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (9,9))
    dilate = cv2.dilate(opening, dilate_kernel, iterations=5)

    cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)

    for c in cnts:
        x,y,w,h = cv2.boundingRect(c)
        cv2.rectangle(mask, (x, y), (x + w, y + h), (36,255,12), 2)
        ROI = result[y:y+h, x:x+w]
        break
    
    if dataset['Tumour_Contour'][i] == '-':
        mask = np.zeros_like(ROI)
        cv2.imwrite('/kaggle/working/mask/'+ name_image_mask[i] + '.jpg', mask)
    else:
        mask = skimage.io.imread(dataset['Tumour_Contour'][i],as_gray=True)
        mask = mask[y:y+h, x:x+w]
        cv2.imwrite('/kaggle/working/mask/'+ name_mask[i], mask)
    
    cv2.imwrite('/kaggle/working/image/'+name_image[i], ROI)